In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# 네이버 블로그 정보 수집하여 저장하기

print("=" *80)
print(" 네이버 VIEW - 저장할 내용을 목록으로 만들어서 xls , csv 형식으로 저장하기")
print("=" *80)

#Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import sys        # system 설정을 변경하기 위해 필요합니다
import math
import pandas  as pd   
import os
import openpyxl

query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
cnt=int(input('2.수집할 데이터는 몇 건입니까?: ') )
page_cnt = math.ceil(cnt / 30)

f_dir = input('3.결과를 저장할 폴더이름을 입력해주세요(기본경로: c:\\py_temp\\) :')
if f_dir =='' :
    f_dir = 'c:\\py_temp\\'
    
#Step 1. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get('http://www.naver.com')
time.sleep(1)

#Step 2. 네이버 검색창에 입력 받은 검색어를 넣고 검색한 후 "블로그" 선택
element = driver.find_element(By.ID ,"query")
element.send_keys(query_txt)
element.submit()

driver.find_element(By.LINK_TEXT,"VIEW").click()
time.sleep(1)

driver.find_element(By.LINK_TEXT,"블로그").click()

# Step 9. 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2 = [ ]           # 게시글 번호 컬럼
title2 = [ ]        # 게시물 제목 컬럼
contents2 = [ ]     # 게시글 내용 컬럼
bdate2 = [ ]        # 작성 일자 컬럼
nick2 = [ ]         # 블로그 닉네임
writer2 =[ ]        # 작성자

#Step 5. 현재 페이지의 내용을 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기
# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(1)

if page_cnt >= 2 :
    i = 1
    while (i < page_cnt + 1):
        scroll_down(driver) 
        i += 1
        print('%s 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^' %i)

print("\n")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

view_list = soup.find('ul','lst_view _list_base').find_all('li')

no = 1

for i in view_list :
    
    no2.append(no)                            # 게시물 번호 리스트에 추가
    print('1.번호:',no)

    title = i.find('div','title_area').find('a').get_text( ) 
    #title = all_title[5].get_text( )          # 게시물 제목
    title2.append(title)                      # 게시물 제목 리스트에 추가
    print('2.제목:',title)

    writer = i.find('div','user_info').find('a').get_text( ) 
    #writer = all_title[4].get_text( )          # 게시물 작성자
    writer2.append(writer)                      # 게시물 작성자 리스트에 추가
    print('3.작성자:',writer)

    
    try :
        contents = i.find('div' , 'dsc_area').get_text( )   # 게시물 내용
    except :
        contents = '본문 요약 내용이 없습니다'
        contents2.append(contents)                # 게시물 내용 리스트에 추가
        print('4.요약내용:',contents)
    else :
        contents2.append(contents)                # 게시물 내용 리스트에 추가
        print('4.요약내용:',contents)

    bdate = i.find('span','sub').get_text( )  # 작성일자
    bdate2.append(bdate)                     # 작성일자 리스트에 추가
    print('5.작성일자:',bdate)

    print("\n")

    if no == cnt :
        break
        
    no += 1

# 출력 결과를 표(데이터 프레임) 형태로 만들기

naver_blog = pd.DataFrame()
naver_blog['번호'] = no2
naver_blog['제목'] = title2
naver_blog['작성자'] = writer2
naver_blog['요약내용'] = contents2
naver_blog['작성일자'] = bdate2

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

sec_name = 'Naver_View'
os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

# csv 형태로 저장하기
naver_blog.to_csv(fc_name,encoding="utf-8-sig",index=False)
print(" csv 파일 저장 경로: %s" %fc_name) 

# 엑셀 형태로 저장하기
import openpyxl
naver_blog.to_excel(fx_name , index=False, engine='openpyxl')
print(" xls 파일 저장 경로: %s" %fx_name) 

# 엑셀파일 실행하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.Dispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
excel.ActiveWorkbook.Save()
excel.Visible=True

driver.close( )

In [ ]:
#Lesson 3.국내 여행 키워드 분석
#Step 1. 필요한 모듈을 실행합니다.
from konlpy.tag import *        #pip install konlpy 먼저 하세요
import matplotlib.pyplot as plt #pip install matplotlib 먼저 하세요
from matplotlib import font_manager , rc
from wordcloud import WordCloud  # pip install wordcloud 먼저 하세요
from collections import Counter
import pandas as pd

okt = Okt()
kkma = Kkma( )

#Step 2 . 텍스트 파일을 불러와서 형태소 분석을 합니다.
data1 = open(fc_name,encoding="UTF-8").read( )
data1 = contents2

#print(data1)
print("\n")

#Step 3. 키워드를 추출합니다
data2=[]
for a in data1 :
    data2.append( okt.nouns(a) )

#Step 4. 추출된 단어들의 빈도를 조사한 후 많이 언급된 100개만 출력합니다
print("\n")
data3 = sum(data2, [])
data4 = Counter(data3)
data5 = data4.most_common(100)

#Step 5. 불용어 제거하기
sword = open("c:\\temp\\여행gsub.txt").read()

data6 = [ each_word for each_word in data3
          if each_word not in sword ]

#용어 정리하기
data6_1 = [ ]
for i in data6 :
    if len(i) >= 2 and len(i) <= 10 :
        if i == '제주' or i == '제주도' or i=='우도' or i =='애월' :
            data6_1.append(i.replace(i,'제주도'))
            
        elif i == '국내' or i == '대한민국' or i == '한국' :
            data6_1.append(i.replace(i,'국내여행'))
        elif i == '핑크' or i == '뮬리' or i == '핑크뮬리' :
            data6_1.append(i.replace(i,'핑크뮬리'))
        elif i == '아이' or i == '어린이' or i == '아기' or i =='키즈':
            data6_1.append(i.replace(i,'아이'))
        elif i == '엘리' or i == '시안' or i == '강촌':
            data6_1.append(i.replace(i,'강촌엘리시안'))
        else :
                data6_1.append(i)

            
#Step 6. 글자수로 불용어 제거하기
data7 = []
for i in data6_1 :
    if len(i) >= 2 and len(i) <= 10 :
        data7.append(i) 

In [ ]:
# Step 7. 단어별 빈도수 집계하기
data8 = Counter(data7)
data9 = data8.most_common(200)
print("2.단어별 빈도수:",data9)
data10 = dict(data9)

In [ ]:
data9_2 = pd.DataFrame(data9)
data9_3 = data9_2.rename(columns={0:'키워드' , 1:'빈도수'})
display(data9_3)
data9_3.to_csv('c:\\py_temp\\가을여행주요키워드.csv' , index=False , encoding='utf-8-sig')
data9_10 = data9_3.head(10)
display(data9_10)

In [ ]:
#바 차트 그리기 - 상위 10군데만
from matplotlib import pyplot as plt 
import matplotlib.font_manager as fm
import matplotlib

font_location = "C:\\Windows\\Fonts\\malgun.ttf"
font_name = fm.FontProperties(fname = font_location).get_name()
matplotlib.rc('font' , family=font_name)
plt.style.use('ggplot')
plt.figure(figsize=(10,6))

x = data9_10['키워드']
y = data9_10['빈도수']

colors = [np.random.rand(3,) for _ in data9_10['키워드']]

plt.bar(x,y , color = colors , width=0.5)
plt.title('가을 여행 주요 키워드')
plt.ylabel('빈도수(단위:건)')
plt.xlabel('키워드')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Step 8. 워드 클라우드 그리기
wordcloud = WordCloud(font_path="c:\\windows\\fonts\\HMKMG.TTF" ,
                       relative_scaling=0.4,
                       background_color="white"
                     ).generate_from_frequencies(data10)
plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np         # pip install numpy
from PIL import Image      # pip install Image
from wordcloud import ImageColorGenerator
korea = np.array(Image.open("c:\\data\\data\\image\\heart.jpg"))
wc = WordCloud(font_path="c:\\windows\\fonts\\HMKMG.TTF" ,
                       relative_scaling=0.2,mask = korea,
                       background_color="white",
                       min_font_size=1,
                       max_font_size=50,
                       max_words=5000
                     ).generate_from_frequencies(data10)
plt.figure(figsize=(8,8))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
korea = np.array(Image.open("c:\\data\\data\\image\\korea.jpg"))
wc = WordCloud(font_path="c:\\windows\\fonts\\HMKMG.TTF" ,
                       relative_scaling=0.2,mask = korea,
                       background_color="white",
                       min_font_size=1,
                       max_font_size=50,
                       max_words=5000
                     ).generate_from_frequencies(data10)
plt.figure(figsize=(5,7))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
korea = np.array(Image.open("c:\\data\\data\image\star.png"))
wc = WordCloud(font_path="c:\\windows\\fonts\\HMKMG.TTF" ,
                       relative_scaling=0.2,mask = korea,
                       background_color="white",
                       min_font_size=1,
                       max_font_size=50,
                       max_words=5000
                     ).generate_from_frequencies(data10)
plt.figure(figsize=(10,8))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
korea = np.array(Image.open("c:\data\\data\image\samsung.jpg"))
wc = WordCloud(font_path="c:\\windows\\fonts\\HMKMG.TTF" ,
                       relative_scaling=0.2,mask = korea,
                       background_color="white",
                       min_font_size=1,
                       max_font_size=100,
                       max_words=5000
                     ).generate_from_frequencies(data10)
plt.figure(figsize=(10,8))
plt.imshow(wc)
plt.axis('off')
plt.show()